# 9. 프로젝트 - Movielens 영화 추천 실습

## Movielens 데이터셋을 활용한 영화 추천시스템
- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. `MovieLens 1M Dataset` 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
- 별점을 **시청횟수**로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

#### 실습 목표
- 추천시스템의 개념과 목적을 이해한다.
- Implicit 라이브러리를 활용하여 Matrix Factorization(이하 MF) 기반의 추천 모델을 만들어 본다.
- 음악 감상 기록을 활용하여 비슷한 아티스트를 찾고 아티스트를 추천해 본다.
- 추천 시스템에서 자주 사용되는 데이터 구조인 CSR Matrix을 익힌다
- 유저의 행위 데이터 중 Explicit data와 Implicit data의 차이점을 익힌다.
- 새로운 데이터셋으로 직접 추천 모델을 만들어 본다.

#### 목차
1. 들어가며
2. 데이터 탐색하기와 전처리
3. 사용자의 명시적/암묵적 평가
4. Matrix Factorization(MF)
5. CSR(Compressed Sparse Row) Matrix
6. MF 모델 학습하기
7. 비슷한 아티스트 찾기 + 유저에게 추천하기
8. 프로젝트 - MovieLens 영화 추천 실습

### 추천시스템이란?
- **협업 필터링(Collaborative Filtering) 방식**  
기존 사용자 행동 정보를 분석하여 해당 사용자와 비슷한 성향의 사용자들이 기존에 좋아했던 항목을 추천하는 기술 (아이템과 사용자 간의 행동 또는 관계에만 주목)
- **콘텐츠 기반 필터링(Contents-based Filtering) 방식**  
항목 자체를 분석하여 항목 간의 유사성을 파악하여 추천을 구현 (아이템 자체의 속성에만 주목)

**협업 필터링을 바로 사용할 수 없게 만드는 세 가지 제약조건**  
1. 콜드 스타트(Cold Start) : 기존의 자료를 기반으로 추천을 제공하므로 기존에 없던 새로운 항목이 추가되는 경우 추천이 곤란해진다
2. 계산량이 비교적 많은 알고리즘이므로 사용자 수가 많은 경우 효율적으로 추천할 수 없다
3. 롱테일(Long tail) : 사용자들의 관심이 적은 다수의 항목은 추천을 위한 충분한 정보를 제공하지 못하는 경우가 많다

## 1) 데이터 준비와 전처리

### 데이터 준비

Movielens 데이터는 `rating.dat` 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 정리되어 있다.

In [1]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings = pd.merge(ratings, movies)
ratings = ratings[["user_id", "movie_id", "title", "count"]]
ratings.sort_values("user_id")

,user_id,movie_id,title,count
0,1,1193,One Flew Over the Cuckoo's Nest (1975),5
31113,1,2294,Antz (1998),4
31674,1,3186,"Girl, Interrupted (1999)",4
32044,1,1566,Hercules (1997),4
32415,1,588,Aladdin (1992),4
...,...,...,...,...
657728,6040,334,Vanya on 42nd Street (1994),4
393446,6040,1294,M*A*S*H (1970),4
253075,6040,994,Big Night (1996),3
127665,6040,2396,Shakespeare in Love (1998),3


## 2) 분석해 봅시다.

In [7]:
# 영화 수
print('ratings에 있는 유니크한 영화 개수 :', ratings['movie_id'].nunique())
# 유저 수
print('ratings에 있는 유니크한 사용자 수 :', ratings['user_id'].nunique())

ratings에 있는 유니크한 영화 개수 : 3628
ratings에 있는 유니크한 사용자 수 : 6039


In [8]:
movie_count = ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

## 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

#### 검색을 쉽게하기 위해 title 문자열 소문자로 바꿔준다.

In [9]:
ratings['title'] = ratings['title'].str.lower()
ratings.head(10)

,user_id,movie_id,title,count
0,1,1193,one flew over the cuckoo's nest (1975),5
1,2,1193,one flew over the cuckoo's nest (1975),5
2,12,1193,one flew over the cuckoo's nest (1975),4
3,15,1193,one flew over the cuckoo's nest (1975),4
4,17,1193,one flew over the cuckoo's nest (1975),5
5,18,1193,one flew over the cuckoo's nest (1975),4
6,19,1193,one flew over the cuckoo's nest (1975),5
7,24,1193,one flew over the cuckoo's nest (1975),5
8,28,1193,one flew over the cuckoo's nest (1975),3
9,33,1193,one flew over the cuckoo's nest (1975),5


In [10]:
ratings

,user_id,movie_id,title,count
0,1,1193,one flew over the cuckoo's nest (1975),5
1,2,1193,one flew over the cuckoo's nest (1975),5
2,12,1193,one flew over the cuckoo's nest (1975),4
3,15,1193,one flew over the cuckoo's nest (1975),4
4,17,1193,one flew over the cuckoo's nest (1975),5
...,...,...,...,...
836473,5851,3607,one little indian (1973),5
836474,5854,3026,slaughterhouse (1987),4
836475,5854,690,"promise, the (versprechen, das) (1994)",3
836476,5938,2909,"five wives, three secretaries and me (1998)",4


In [12]:
my_favorite = ['matrix, the (1999)' , 'terminator 2: judgment day (1991) ' ,'shining, the (1980)' ,'clockwork orange, a (1971)' ,'400 blows, the (les quatre cents coups)']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_movielist = pd.DataFrame({'user_id': ['wooil']*5, 'title': my_favorite, 'movie_id': my_favorite, 'count':[5]*5})

if not ratings.isin({'user_id':['wooil']})['user_id'].any():
    ratings = ratings.append(my_movielist)

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,title,count
836473,5851,3607,one little indian (1973),5
836474,5854,3026,slaughterhouse (1987),4
836475,5854,690,"promise, the (versprechen, das) (1994)",3
836476,5938,2909,"five wives, three secretaries and me (1998)",4
836477,5948,1360,identification of a woman (identificazione di ...,5
0,wooil,"matrix, the (1999)","matrix, the (1999)",5
1,wooil,terminator 2: judgment day (1991),terminator 2: judgment day (1991),5
2,wooil,"shining, the (1980)","shining, the (1980)",5
3,wooil,"clockwork orange, a (1971)","clockwork orange, a (1971)",5
4,wooil,"400 blows, the (les quatre cents coups)","400 blows, the (les quatre cents coups)",5


In [13]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [14]:
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna() # .dropna() : 결측치(누락데이터) 제거
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 주기
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 주기
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie id column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie id column indexing Fail!!')

ratings


user_id column indexing OK!!
movie id column indexing OK!!


,user_id,movie_id,title,count
0,0,0,one flew over the cuckoo's nest (1975),5
1,1,0,one flew over the cuckoo's nest (1975),5
2,2,0,one flew over the cuckoo's nest (1975),4
3,3,0,one flew over the cuckoo's nest (1975),4
4,4,0,one flew over the cuckoo's nest (1975),5
...,...,...,...,...
0,6039,3628,"matrix, the (1999)",5
1,6039,3629,terminator 2: judgment day (1991),5
2,6039,3630,"shining, the (1980)",5
3,6039,3631,"clockwork orange, a (1971)",5


In [ ]:
ratings.head(30)

### MF model 구성하기

## 3. Matrix Factorization(MF)
데이터가 준비되었으므로 이제 모델을 만든다.  

**1. m명의 사용자들이 n명의 아티스트에 대해 평가한 (m,n) 사이즈의 평가행렬(Rating Matrix)**  
- 행렬 중 일부는 데이터가 채워져 있지만, 나머지 부분은 데이터가 비어 있는데, 추천시스템의 `협업 필터링(Collaborative Filtering)`이란 이런 평가행렬을 전제로 한다.
- 이 평가행렬의 비어있는 부분을 포함한 완벽한 정보를 얻을 수 있다면, 완벽한 추천이 가능해진다

**2. 추천 시스템의 다양한 모델 중 `Matrix Factorization(MF, 행렬분해)` 모델을 사용한다.**  
- (m,n) 사이즈의 행렬 R을 (m,k) 사이즈의 행렬 P와 (k,n) 사이즈의 행렬 Q로 분해한다면 R이란 그저 P와 Q의 행렬곱으로 표현 가능할 수 있다는 간단한 아이디어
-  아이디어의 단순함에도 불구하고 MF 모델은 성능이 준수하고 Scalability가 좋아서 많이 사용되는 모델

## 4. CSR(Compressed Sparse Row) Matrix
모델 학습의 input으로 사용할 데이터 타입을 `CSR(Compressed Sparse Row) Matrix`로 한다.  
CSR Matrix는 Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보만으로 구성하여 메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조

#### 이전 스텝에서 설명한 Matrix Factorization 모델을 implicit 패키지를 사용하여 학습해본다.
- `implicit` 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지이다
- 이 패키지에 구현된 `als(AlternatingLeastSquares)` 모델을 사용한다. `Matrix Factorization`에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적인 것으로 알려져 있다.

**AlternatingLeastSquares 클래스의 __init__ 파라미터**  
1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인가
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인가
3. use_gpu : GPU를 사용할 것인가
4. iterations : epochs와 같은 의미

데이터를 몇 번 반복해서 학습할 것인지 1,4를 늘릴수록 학습데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 한다.

모델 학습 끝! 이제 2가지를 아래 사항을 살펴본다.

1. wooil 벡터와 deep purple의 벡터를 어떻게 만들고 있는가
2. 두 벡터를 곱하면 어떤 값이 나오는가

## 5. 비슷한 아티스트 찾기 + 유저에게 추천하기

### 비슷한 아티스트 찾기
`AlternatingLeastSquares` 클래스에 구현되어 있는 `similar_items` 메서드를 통하여 비슷한 아티스트를 찾는다

### 유저에게 아티스트 추천하기
`AlternatingLeastSquares` 클래스에 구현되어 있는 `recommend` 메서드를 통해 아티스트를 추천받는다.  
`filter_already_liked_items` 는 유저가 이미 평가한 아이템은 제외하는 Argument이다.

`AlternatingLeastSquares` 클래스에 구현된 `explain` 메소드를 사용하면 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인할 수 있다.

||  <center>**평가 문항**</center> |  <center>**상세 기준**</center> |
|:--------|:--------|:--------|
|**1**|CSR matrix가 정상적으로 만들어졌다.|사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.|
|**2**|MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.|사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.|
|**3**|비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.|MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.|